---

## EXERCISE1: Burglary network

Implement the *Prior Sampling* algorithm to do approximate inference on last week's Burglary network.

<img src='https://drive.google.com/uc?id=11xqbchd4TCFSzdLVxNgr4SoeGfrSoqGO'>

Verify that the algorithm can correctly approximate the probability $P(j, m, a, \neg b, \neg e) = 0.00063$

Try different numbers of samples (e.g. $N = 10, 100, 1000, 10000$) and compare the results.

---

In [ ]:
# I report the code explained during the lesson below since it is the base for solving the exercise 1
import numpy as np
import random as rnd

t, f = 0, 1

def samplegen(Pdist, Parents = []):
	assert len(Parents) < len(Pdist.shape)
	if rnd.random() < Pdist[t][tuple(Parents)]:
		return t
	return f

def parents(X):
	return [val[i] for i in par[X]]


# Given the request to apply prior sampling with different number of samples, I create a function with the code explained during the lesson and with the param
#the number of events
def priorSampling(n_events):
  event = []

  for n in range(n_events):
    for x in var:
      val[x] = samplegen(prd[x], parents(x))
    event.append(['f' if val[x] else 't' for x in var])

  print("Number or randomly generated events = ", len(event))
  return event


################################################################################

# Now Let's define the network using the same structures presented during the lesson


# problem distributions
P_B = np.array([0.001, 0.999])
P_E = np.array([0.002, 0.998])
P_A_BE = np.array([[[0.95, 0.94],[0.29, 0.001]],[[0.05, 0.06],[0.71, 0.999]]])
P_J_A = np.array([[0.9, 0.05],[0.1, 0.95]])
P_M_A = np.array([[0.7, 0.01],[0.3, 0.99]])


# network variables...
var = ['B','E','A','J','M']
# their distributions...
prd = {'B':P_B, 'E':P_E, 'A':P_A_BE, 'J':P_J_A, 'M':P_M_A}
# their parents...
par = {'B':[],'E':[], 'A':['B','E'], 'J':['A'], 'M':['A']}
# and their initial values
val = {'B':f, 'E':f, 'A':f, 'J':f, 'M':f}



In [ ]:
#Q.1.1 Try different numbers of samples (e.g. $N = 10, 100, 1000, 10000$) and compare the results.
test_values = [10**(i+1) for i in range(6)] #a loop based on the power of 10
for i in range(len(test_values)):
  data = priorSampling(test_values[i]) # generate the samples
  P_query = data.count(['f', 'f', 't', 't', 't']) / len(data) # we have to compute P(j,m,a,¬b,¬e), pay attention to the order of the variables
  print("P(j,m,a,¬b,¬e) = ", P_query)

Number or randomly generated events =  10
P(j,m,a,¬b,¬e) =  0.0
Number or randomly generated events =  100
P(j,m,a,¬b,¬e) =  0.0
Number or randomly generated events =  1000
P(j,m,a,¬b,¬e) =  0.001
Number or randomly generated events =  10000
P(j,m,a,¬b,¬e) =  0.0009
Number or randomly generated events =  100000
P(j,m,a,¬b,¬e) =  0.00066
Number or randomly generated events =  1000000
P(j,m,a,¬b,¬e) =  0.000663


---

## EXERCISE2: Pomegranate for Day 2

Use <tt>pomegranate</tt> to calculate the filtered probability of rain on Day 2 when we see an umbrella on Day 1 and Day 2.

What is the filtered probability of rain on Day 2 when we see <tt>not umbrella</tt> on Day 1?

How about if we just have no information about Umbrellas on Day 1 (i.e., only that rain on Day 2)?

In [ ]:
# I report the code explained during the lesson below since it is the base for solving the exercise 2
#!pip install numpy==1.23.0 --ignore-installed
#!pip install pomegranate==0.14.6
!pip install numpy #1.25.2
#!pip install pomegranate==0.14.7
!pip install pomegranate==0.15.0
from pomegranate import *

# Variables are RainN and UmbrellaN+1 for N = 0, 1, ...
# We have a prior for Rain0, two values 'y'es and 'n'o:
Rain0   = DiscreteDistribution({'y': 0.5, 'n': 0.5})

# Transition model
#
# Conditional distribution relating RainN and RainN+1. Notation for
# the conditional probability table is:
#
# [ 'RainN', 'RainN+1', <probability>]
#
# for the conditional value P(Sprinkler|Cloudy). Note that we have to
# repeat the transition model for each pair of states
Rain1 = ConditionalProbabilityTable(
        [['y', 'y', 0.7],
         ['y', 'n', 0.3],
         ['n', 'y', 0.3],
         ['n', 'n', 0.7]], [Rain0])

Rain2 = ConditionalProbabilityTable(
        [['y', 'y', 0.7],
         ['y', 'n', 0.3],
         ['n', 'y', 0.3],
         ['n', 'n', 0.7]], [Rain1])

# Sensor model
#
# Conditional distribution relating Rain and Umbrella:
#
# [ 'Umbrella', 'Rain', <probability>]
#
# for the conditional value P(Sprinkler|Cloudy). Values for Umbrella are 'y'es and 'n'o.
# Again we have to enter the table for each day.
Umbrella1 = ConditionalProbabilityTable(
        [['y', 'y', 0.9],
         ['y', 'n', 0.1],
         ['n', 'y', 0.2],
         ['n', 'n', 0.8]], [Rain1])

Umbrella2 = ConditionalProbabilityTable(
        [['y', 'y', 0.9],
         ['y', 'n', 0.1],
         ['n', 'y', 0.2],
         ['n', 'n', 0.8]], [Rain2])
#
# The whole network has five nodes:
s1 = Node(Rain0, name="Rain0")
s2 = Node(Rain1, name="Rain1")
s3 = Node(Umbrella1, name="Umbrella1")
s4 = Node(Rain2, name="Rain2")
s5 = Node(Umbrella2, name="Umbrella2")
# Create a network that includes nodes and edges between them:
model = BayesianNetwork("Umbrella Network")
model.add_states(s1, s2, s3, s4, s5)
model.add_edge(s1, s2)
model.add_edge(s2, s3)
model.add_edge(s2, s4)
model.add_edge(s4, s5)
# Fix the model structure
model.bake()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 34.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pomegranate: filename=pomegranate-0.15.0-cp310-cp310-linux_x86_64.whl size=21355167 sha256=806b74c219e254dfdfa3d9d987b17a8e2f43ac466d8849e4e11ab9526ca428e4
  Stored in directory: /root/.cache/pip/wheels/55/44/b9/c0399e42f50907ada5da7d160bcd1465a39f36b935d9e59f5d
Successfully built pomegranate


In [ ]:
# Conditional probability table
Rain3 = ConditionalProbabilityTable(
        [['y', 'y', 0.7],
         ['y', 'n', 0.3],
         ['n', 'y', 0.3],
         ['n', 'n', 0.7]], [Rain2])
# Node
s6 = Node(Rain3, name="Rain3")
# State
model.add_states(s6)
# Edge
model.add_edge(s4, s6)
# Fix the model structure
model.bake()

Q2.1 Use <tt>pomegranate</tt> to calculate the filtered probability of rain on Day 2 when we see an umbrella on Day 1 and Day 2.

In [ ]:
#QUESTION 2.1
# Set Umbrella1 and Umbrella 2 to 'y'
scenario = [[None, None, 'y', None, 'y', None]]
# Run the model
results =  model.predict_proba(scenario)
# And the probability of rain on Day 2:
print(results[0][3].items())

(('y', 0.8833570412517777), ('n', 0.11664295874822232))


Q2.2 What is the filtered probability of rain on Day 2 when we see <tt>not umbrella</tt> on Day 1? (But we still see it in Day 2)

In [ ]:
#QUESTION 2.2
# Don't see Umbrella1 on Day 1 (i.e., set to 'n'), set Umbrella 2 to 'y'
scenario = [[None, None, 'n', None, 'y', None]]
# Run the model
results =  model.predict_proba(scenario)
# Ask for the probability of rain on Day 2:
print(results[0][3].items())

(('y', 0.7027707808564231), ('n', 0.2972292191435768))


Q2.3 How about if we just have no information about Umbrellas on Day 1 (But we still see the umbrella in Day 2)?

In [ ]:
# QUESTION 2.3
# No infos about umbrellas only Rain on day2 equal
scenario = [[None, None, None, None, 'y', None]]
# Run the model
results =  model.predict_proba(scenario)
#print(results)
# Ask for the probability of rain on Day 2:
print(results[0][3].items())

(('y', 0.8181818181818179), ('n', 0.1818181818181821))
